This little program enables the autoimation of grade collection, checking for upcoming due dates and maybe something else if I get a request or an idea. 
Requirements:
Selenium: https://pypi.org/project/selenium/
Numpy: https://pypi.org/project/numpy/
Pandas: https://pypi.org/project/pandas/


This sets up the webdriver. This only works in firefox HOWEVER you can use chrome using this instead: 
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options)
**note that I havent tested it in chrome. not all web drivers are the same so your milage may vary. 

In [138]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

options = webdriver.FirefoxOptions()
driver = webdriver.Firefox(options=options)
options.add_argument("-headless")
options.add_argument('--no-sandbox')

This here sets up the base url of brightspace "https://schoolspecific.brightspace.com" is the typical url base. It also sets up the various buttons that need to be clicked though to get through MSFT login. the getPass import is a input that is passed to your password variable in order to keep it out of the source. 

In [139]:
import getpass # so you don't show your password in the sourcecode
email = 'w0499673@campus.nscc.ca'
base_url = 'https://nscconline.brightspace.com'
password = getpass.getpass()
email_field = (By.ID, 'i0116')
password_field = (By.ID, 'i0118')
next_button = (By.ID, 'idSIButton9')

This just runs through the MSFT login process inorder to get to the brightspace homepage

In [140]:
driver.get(base_url)
WebDriverWait(driver,10).until(EC.element_to_be_clickable(email_field)).send_keys(email)
WebDriverWait(driver,10).until(EC.element_to_be_clickable(next_button)).click()
WebDriverWait(driver,10).until(EC.element_to_be_clickable(password_field)).send_keys(password)
WebDriverWait(driver,10).until(EC.element_to_be_clickable(next_button)).click()
WebDriverWait(driver,10).until(EC.element_to_be_clickable(next_button)).click()

This block Iterates through a list of validated grade page urls. Nested loop then extracts the <table> contents from the HTML on each page. the second for loop removes the "Points" and the "comments and asscessments" column from each list dataframe.

In [141]:
grades_extensions = ["/d2l/lms/grades/my_grades/main.d2l?ou=299885",#Networking
                     "/d2l/lms/grades/my_grades/main.d2l?ou=297335",#d
                     "/d2l/lms/grades/my_grades/main.d2l?ou=295501",#295501
                     "/d2l/lms/grades/my_grades/main.d2l?ou=295683",#WEBDEV
                     "/d2l/lms/grades/my_grades/main.d2l?ou=297213"#Prog
                    ]
import pandas as pd
import io
import numpy as np
import os
from bs4 import BeautifulSoup as bs4
url = []
df = []
new_df=[]
#Iterates through list of grade page urls. Nested loop then extractss the <table> contents from the HTML on each page
for each in grades_extensions:
    url = base_url + each
    driver.get(url)
    soup = bs4(driver.page_source, 'html.parser')
    soup = io.StringIO(str(soup))
    df = pd.read_html(soup)

  
    #This clears out the columns "Points" and "Comments and Assessments". 
    for i in df:
        i.pop("Points")
        i.pop("Comments and Assessments")
        new_df.append(i)
print(new_df)   
driver.close()


[              Grade Item          Grade Item.1 Weight Achieved  Grade
0     Weekly Assignments    Weekly Assignments      38.67 / 40    NaN
1                    NaN          Assignment 1   13.33 / 13.33    NaN
2                    NaN          Assignment 2   13.33 / 13.33    NaN
3                    NaN          Assignment 3      12 / 13.33    NaN
4                    NaN          Assignment 4           - / -    NaN
5                    NaN          Assignment 5           - / -    NaN
6                    NaN  Assignment 6 (bonus)           - / -    NaN
7         Weekly Quizzes        Weekly Quizzes      35.73 / 40    NaN
8                    NaN                Quiz 1    11.2 / 13.33    NaN
9                    NaN                Quiz 2    12.8 / 13.33    NaN
10                   NaN                Quiz 3   11.73 / 13.33    NaN
11                   NaN                Quiz 4           - / -    NaN
12                   NaN                Quiz 5           - / -    NaN
13                 

This block splits each course dataframe into separate dataframes and writes each to a .csv file for easy convenience on checking for data.

In [142]:
csv_file_folder = "./CsvFiles/"
try:
    os.mkdir(csv_file_folder)

    #Networking
    df0 = new_df[0]
    if os.path.isfile("./CsvFiles/Networking.csv") == True:
        print(f'./CsvFiles/Networking.csv already exists. Moving on...')
    else:
        df0.to_csv("./CsvFiles/Networking.csv",index=False)

    #DATA FUND
    df1 = new_df[1]
    if os.path.isfile("./CsvFiles/DataFund.csv") == True:
        print(f'./CsvFiles/DataFund.csv already exists. Moving on...')
    else:
        df1.to_csv("./CsvFiles/DataFund.csv",index=False)


    #OSYS
    df2 = new_df[2]#OSYS
    if os.path.isfile("./CsvFiles/Osys.csv") == True:
        print(f'./CsvFiles/Osys.csv already exists. Moving on...')
    else:
        df2.to_csv("./CsvFiles/Osys.csv",index=False)


    #WEBDEV
    df3 = new_df[3]#
    if os.path.isfile("./CsvFiles/Osys.csv") == True:
        print(f'./CsvFiles/Webdev.csv already exists. Moving on...')
    else:
        df3.to_csv("./CsvFiles/Webdev.csv",index=False)

    #Programming  
    df4 = new_df[4]
    if os.path.isfile("./CsvFiles/Prog.csv") == True:
        print(f'./CsvFiles/Prog.csv already exists. Moving on...')
    else:
        df4.to_csv("./CsvFiles/Prog.csv",index=False)

    #Concatenating all dataframes to a csv file. Not super needed but helps to view the data. 
    if os.path.isfile("./CsvFiles/parent.csv") == True:
        print(f'./CsvFiles/parent.csv already exists. Moving on...')
    else: 
        pd.concat([df0,df1,df2,df3,df4], axis=1).to_csv("./CsvFiles/parent.csv", index=False)
except FileExistsError:
    print(f'Exiting..')

./CsvFiles/Webdev.csv already exists. Moving on...
